In [1]:
%pip install google-generativeai langgraph tavily
%pip install tavily-python
from tavily import TavilyClient

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily


Note: you may need to restart the kernel to use updated packages.
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\cadup\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
import os
import google.generativeai as genai

from dotenv import load_dotenv

load_dotenv()


os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY') 
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')


In [3]:
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

result = client.search("O que são multiagentes de Inteligência Artificial?",
                       include_answer=True)

result["answer"]

'Multiagentes de Inteligência Artificial são sistemas compostos por múltiplos agentes autônomos que colaboram para resolver problemas. Cada agente pode agir independentemente, mas trabalha em conjunto com outros para alcançar objetivos comuns. Eles melhoram o desempenho e eficiência do sistema.'

In [4]:
cidade = "Belém"

query = f"""
Liste os 5 principais restaurantes em {cidade}, segundo avaliações recentes no TripAdvisor ou sites similares de turismo.
Para cada restaurante, informe:
- Tipo de culinária (ex: regional, italiana, japonesa)
- Uma breve descrição (máx. 2 linhas)
- Avaliação média (se disponível)
- Faixa de preço

Responda apenas com dados atualizados e relevantes para turistas.
"""

In [5]:
%pip install -U duckduckgo_search

Note: you may need to restart the kernel to use updated packages.


In [6]:
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        raise e

for link in search(query):
    print(link)

C:\Users\cadup\AppData\Local\Temp\ipykernel_27268\3834310468.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


https://www.tripadvisor.com/Restaurants-g303404-Belem_State_of_Para.html
https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html
https://exame.com/casual/os-melhores-restaurantes-de-belem-segundo-ranking-exame-casual-2025/
https://hoteisrestaurantes.com.br/artigos/melhores-restaurantes-em-belem/
https://www.tripadvisor.co/Restaurants-g303404-Belem_State_of_Para.html
https://restaurantguru.com.br/Belem


In [7]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def scrape_restaurantes_info(url):
    if not url:
        print("Erro: URL vazia ou não localizada.")
        return None

    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8'
}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao carregar a página {url}: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

search_results = search(query)
if search_results:
    url = search_results[0]
else:
    url = None

soup = scrape_restaurantes_info(url)

print(f"Website: {url}\n\n")

if soup:
    print(str(soup.body.prettify())[:50000])
else:
    print("Não foi possível raspar o conteúdo da página ou a URL não foi encontrada.")

C:\Users\cadup\AppData\Local\Temp\ipykernel_27268\328789785.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


Website: https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html


<body>
 <div id="lithium-root">
  <!--$-->
  <div aria-hidden="true" class="adsbygoogle ad-slot ad-banner adsbox ad" id="ad-banner" style="position:absolute;top:0px;bottom:-20px;width:10px;height:10px;background-color:transparent">
   
  </div>
  <!--$-->
  <!--/$-->
  <div class="CvDIt _Q f e Gi Ra _s P6">
   <button class="UikNM _G B- _S _W _T c G_ wSSLS wnNQG LbPqG" type="button">
    <span class="biGQs _P ezezH">
     Pular para o conteúdo principal
    </span>
   </button>
  </div>
  <div class="ad SxZAK f _h u e j Gi">
   <div class="ktyWU f _h u e j w Gi">
    <div class="WMKsc w xbgrb o" id="slot:5x1-728x90:horizon">
    </div>
   </div>
  </div>
  <span>
   <div>
   </div>
   <header class="aVAoX t z">
    <div class="IDaDx Iwmxp mvTrV cyIij fluiI SMjpI">
     <nav class="MGToN u">
      <div class="oqeTs u JpEOb">
       <a class="f _Z wSSLS" href="/">
        <picture>
         <img alt="T

In [8]:

restaurante_text_data = []

for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    if tag.name == 'h1' and 'restaurantes' in tag.get_text(" ", strip=True).lower():
        restaurante_text_data.append(f"Título da Página: {tag.get_text(' ', strip=True)}")

    elif tag.name in ['h2', 'h3'] or (tag.name == 'p' and 'destaque' in tag.get('class', [])):
        restaurante_text_data.append(f"Nome/Destaque: {tag.get_text(' ', strip=True)}")
        
    elif tag.name == 'p':
        
        if len(tag.get_text(" ", strip=True)) > 50:
            restaurante_text_data.append(f"Conteúdo: {tag.get_text(' ', strip=True)}")

final_restaurante_data = "\n".join(restaurante_text_data)

final_restaurante_data = re.sub(r'\s+', ' ', final_restaurante_data).strip()

print(f"Website: {url}\n\n")
print(final_restaurante_data)

Website: https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html


Título da Página: Melhores restaurantes: Belém, PA - 2025 Nome/Destaque: Restaurantes: Belém Nome/Destaque: Tipo Nome/Destaque: Tipo de refeição Nome/Destaque: Culinárias Nome/Destaque: Pratos Nome/Destaque: Prêmios Os vencedores do Prêmio Travellers’ Choice (incluindo o título “Os melhores dos melhores”) estão entre os 10% dos melhores perfis no Tripadvisor, de acordo com avaliações e opiniões de viajantes do mundo todo. Nome/Destaque: Preço Nome/Destaque: Pontuação Nome/Destaque: Aberto agora Nome/Destaque: Restrições alimentares Nome/Destaque: Excelente para Nome/Destaque: Características Nome/Destaque: Bairro Conteúdo: A Tripadvisor LLC não dá garantias da disponibilidade das tarifas anunciadas no nosso site e aplicativos. Os preços de tabela podem exigir uma estadia com uma determinada duração, ter datas bloqueadas, qualificações ou outras restrições aplicáveis. A Tripadvisor LLC não é responsáv

In [9]:
import os
import re
from tavily import TavilyClient

api_key = os.environ.get("TAVILY_API_KEY")
if not api_key:
    raise ValueError("A chave TAVILY_API_KEY não foi encontrada.")

cliente_tavily = TavilyClient(api_key=api_key)

cidade = "Belém do Pará"
tavily_query = f"restaurantes em {cidade} tripadvisor com maior quantidade de reviews e faixa de preço"


print("Iniciando busca agêntica por URLs do Tripadvisor com Tavily...")
tripadvisor_url = None
try:
    tavily_results = cliente_tavily.search(query=tavily_query, max_results=5)

    if tavily_results and tavily_results["results"]:
        print(f"Tavily encontrou {len(tavily_results['results'])} resultados. Analisando...")

        for result in tavily_results["results"]:
            url = result["url"]
            
            if "tripadvisor.com" in url or "tripadvisor.com.br" in url:
                tripadvisor_url = url
                break

        if not tripadvisor_url:
            print("Nenhum URL relevante do Tripadvisor foi encontrado nos primeiros resultados.")
            
    else:
        print("Tavily não encontrou resultados para a busca agêntica.")

except Exception as e:
    print(f"Erro na busca agêntica com Tavily: {e}. Verifique sua chave API ou conexão.")

if tripadvisor_url:
    clean_url = re.sub(r'-oa\d+-', '-', tripadvisor_url)
    tripadvisor_url = clean_url
    print(f"  ✅ URL encontrada limpa de paginação.")

print("-" * 50)
print(f"URL Final do Tripadvisor para raspagem: {tripadvisor_url if tripadvisor_url else 'NÃO ENCONTRADO'}")
print("-" * 50)

Iniciando busca agêntica por URLs do Tripadvisor com Tavily...
Tavily encontrou 5 resultados. Analisando...
  ✅ URL encontrada limpa de paginação.
--------------------------------------------------
URL Final do Tripadvisor para raspagem: https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html
--------------------------------------------------


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import WebDriverException, TimeoutException

def scrape_restaurantes_info(url):
    if not url:
        print("Erro: URL vazia ou não localizada para raspagem.")
        return None

    try:
        service = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(30)
        
    except Exception as e:
        print(f"Erro ao inicializar o driver do Selenium: {e}")
        return None

    try:
        print(f"Tentando carregar a página com Selenium: {url}")
        driver.get(url)
        
        driver.implicitly_wait(10)

        response_text = driver.page_source

    except TimeoutException:
        print(f"Erro de tempo limite ao carregar a página {url}.")
        return None
    except WebDriverException as e:
        print(f"Erro ao carregar a página {url} com Selenium: {e}. Pode ser um bloqueio ou problema de conexão.")
        return None
    finally:
        driver.quit()

    soup = BeautifulSoup(response_text, 'html.parser')
    return soup

soup_tripadvisor = None

if 'tripadvisor_url' in locals() and tripadvisor_url:
    print(f"\nTentando raspar a página identificada: {tripadvisor_url}")
    soup_tripadvisor = scrape_restaurantes_info(tripadvisor_url)

    if soup_tripadvisor:
        print("HTML da página do Tripadvisor obtido com sucesso!")
        page_title_tag = soup_tripadvisor.find('title')
        if page_title_tag:
            print(f"Título da página: {page_title_tag.get_text(strip=True)}")
        else:
            print("Não foi possível encontrar o título da página.")
    else:
        print("Falha ao raspar a página do Tripadvisor. Verifique o URL ou se o site está bloqueando.")
else:
    print("Não há URL do Tripadvisor válido para raspar (obtido no Bloco 1).")

print("-" * 50)


Tentando raspar a página identificada: https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html
Tentando carregar a página com Selenium: https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html
HTML da página do Tripadvisor obtido com sucesso!
Título da página: tripadvisor.com.br
--------------------------------------------------


In [11]:
from bs4 import BeautifulSoup
import re # Importar regex para extração mais limpa

# Este é o HTML completo que você me forneceu - Como se estivessemos ensinando o agente de busca a visualizar a página
html_completo_exemplos = """
<div class="IDaDx Iwmxp mvTrV cyIij fluiI SMjpI"><div class="u f k mowmC"><div class="USSyL" data-test-target="main_h1"><h2 class="biGQs _P fiohW eTlmC">Restaurantes: Belém</h2></div></div><div class="u f k tyUdl"><div class="unvTU"><span class="biGQs _P fiohW RnSgX">Melhores restaurantes em Belém</span><div class="gsRoG NK _Q"><div class="DgJpu"><button type="button" class="ypcsE _S bYExr" aria-labelledby=":lithium-r25i:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-r25i:" class="KrpDA o">Restaurantes classificados de acordo com visualizações de página, avaliações e atributos individuais, como faixa de preço, culinária e localização, bem como dados agregados do Tripadvisor que comparam as interações dos usuários com todas as outras páginas de restaurantes neste local.</span></div></div></div><div class="Mjzwz u f K"><span data-automation="mapButton"><button class="EAeYu z Xa q Pc PQ Pp PD W _S Gy Rd Cr _M pGzPy bYExr ncKoe" type="button" aria-label="Ver mapa"><span class="sZLBj _Y k Q1 ncKoe"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg><span class="biGQs _P AWdfh">Mapa</span></span></button></span></div></div><div class="rxvcc"><div class="seiBA _T wCxWg" data-automation="LeftRailContainer"><div class="wjLQL _T Ft" data-automation="LeftRail"><div class="vILDq e fbkSn"><div class="WVPPt S5 Gi R4 f e EUDqa"><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Tipo</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r25j:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r25j:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.10591" aria-labelledby="establishmentTypes.10591_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="true" name="establishmentTypes.10591" value="10591" checked=""></div><div class="hTbJa f H o"><label id="establishmentTypes.10591_label" for="establishmentTypes.10591" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Restaurantes</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.9909" aria-labelledby="establishmentTypes.9909_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="establishmentTypes.9909" value="9909"></div><div class="hTbJa f H o"><label id="establishmentTypes.9909_label" for="establishmentTypes.9909" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Docerias</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.9900" aria-labelledby="establishmentTypes.9900_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="establishmentTypes.9900" value="9900"></div><div class="hTbJa f H o"><label id="establishmentTypes.9900_label" for="establishmentTypes.9900" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Café e chá</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.11776" aria-labelledby="establishmentTypes.11776_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="establishmentTypes.11776" value="11776"></div><div class="hTbJa f H o"><label id="establishmentTypes.11776_label" for="establishmentTypes.11776" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Bares e pubs</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button" aria-expanded="false" aria-controls="filter-expando-establishmentTypes"><span class="biGQs _P ezezH">Mostrar mais<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Tipo de refeição</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r25o:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r25o:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10597" aria-labelledby="mealTypes.10597_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10597" value="10597"></div><div class="hTbJa f H o"><label id="mealTypes.10597_label" for="mealTypes.10597" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Café da manhã</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10606" aria-labelledby="mealTypes.10606_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10606" value="10606"></div><div class="hTbJa f H o"><label id="mealTypes.10606_label" for="mealTypes.10606" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Brunch</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10598" aria-labelledby="mealTypes.10598_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10598" value="10598"></div><div class="hTbJa f H o"><label id="mealTypes.10598_label" for="mealTypes.10598" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Almoço</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10599" aria-labelledby="mealTypes.10599_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10599" value="10599"></div><div class="hTbJa f H o"><label id="mealTypes.10599_label" for="mealTypes.10599" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Jantar</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Culinárias</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r25t:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r25t:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10348" aria-labelledby="cuisines.10348_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10348" value="10348"></div><div class="hTbJa f H o"><label id="cuisines.10348_label" for="cuisines.10348" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Brasileira</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10749" aria-labelledby="cuisines.10749_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10749" value="10749"></div><div class="hTbJa f H o"><label id="cuisines.10749_label" for="cuisines.10749" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Sul-americana</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10641" aria-labelledby="cuisines.10641_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10641" value="10641"></div><div class="hTbJa f H o"><label id="cuisines.10641_label" for="cuisines.10641" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Pizza</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10670" aria-labelledby="cuisines.10670_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10670" value="10670"></div><div class="hTbJa f H o"><label id="cuisines.10670_label" for="cuisines.10670" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Pub</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button"><span class="biGQs _P ezezH">Mostrar tudo</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Pratos</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r262:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r262:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.23001" aria-labelledby="dishes.23001_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.23001" value="23001"></div><div class="hTbJa f H o"><label id="dishes.23001_label" for="dishes.23001" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Camarão VG</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.10647" aria-labelledby="dishes.10647_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.10647" value="10647"></div><div class="hTbJa f H o"><label id="dishes.10647_label" for="dishes.10647" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Lanches</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.10893" aria-labelledby="dishes.10893_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.10893" value="10893"></div><div class="hTbJa f H o"><label id="dishes.10893_label" for="dishes.10893" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Caranguejo</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.20336" aria-labelledby="dishes.20336_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.20336" value="20336"></div><div class="hTbJa f H o"><label id="dishes.20336_label" for="dishes.20336" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Bife com batata frita</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button"><span class="biGQs _P ezezH">Mostrar tudo</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH"><div class="f">Prêmios<div class="TtrFj"><div class="biGQs _P ezezH"><span class="NK"><div class="DgJpu"><button type="button" class="ypcsE _S bYExr" aria-labelledby=":lithium-r268:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-r268:" class="KrpDA o">Os vencedores do Prêmio Travellers’ Choice (incluindo o título “Os melhores dos melhores”) estão entre os 10% dos melhores perfis no Tripadvisor, de acordo com avaliações e opiniões de viajantes do mundo todo.</span></div></span></div></div></div></h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r267:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r267:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="travelersChoice.coe" aria-labelledby="travelersChoice.coe_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="travelersChoice.coe" value="coe"></div><div class="hTbJa f H o"><label id="travelersChoice.coe_label" for="travelersChoice.coe" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="f Q1 u"><div class="siujn Rb Ce pVgoX"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="M12 3.953a7.442 7.442 0 1 0 .001 14.884A7.442 7.442 0 0 0 12 3.953m0 14.05a6.61 6.61 0 1 1 0-13.218 6.61 6.61 0 0 1 0 13.219M10.343 11.9a.91.91 0 1 1-1.821 0 .91.91 0 0 1 1.821 0m5.134 0a.91.91 0 1 1-1.821 0 .91.91 0 0 1 1.82 0m.82-1.897.84-.913h-1.863A5.8 5.8 0 0 0 12 8.08a5.77 5.77 0 0 0-3.27 1.008H6.862l.84.913a2.567 2.567 0 1 0 3.475 3.78l.823.896.823-.895a2.568 2.568 0 1 0 3.474-3.78m-6.865 3.634a1.738 1.738 0 1 1 0-3.476 1.738 1.738 0 0 1 0 3.476M12 11.85c0-1.143-.832-2.124-1.929-2.543A5 5 0 0 1 12 8.92a5 5 0 0 1 1.928.386c-1.096.42-1.927 1.4-1.927 2.543m2.566 1.787a1.738 1.738 0 1 1 .001-3.476 1.738 1.738 0 0 1 0 3.476m-8.456 3.719s-.377-.946-1.396-1.903c-1.02-.957-2.303-1.132-2.303-1.132s.457 1.02 1.54 2.04c1.086 1.017 2.159.995 2.159.995m2.568 1.41s-.524-.511-1.479-.883-1.861-.191-1.861-.191.598.54 1.615.935c1.016.397 1.725.139 1.725.139m2.493.505s-.545-.224-1.357-.196-1.415.47-1.415.47.608.222 1.473.193 1.3-.467 1.3-.467m-6.186-4.203s-.175-1.008-.974-2.154c-.8-1.147-2.015-1.578-2.015-1.578s.238 1.098 1.089 2.319c.85 1.22 1.9 1.413 1.9 1.413m-1.003-3.071s.195-1.021-.134-2.393c-.328-1.371-1.294-2.21-1.294-2.21s-.17 1.128.18 2.589c.35 1.46 1.248 2.014 1.248 2.014"></path><path d="M17.887 17.355s.377-.946 1.396-1.903c1.02-.957 2.303-1.132 2.303-1.132s-.457 1.02-1.54 2.04c-1.086 1.017-2.159.995-2.159.995m-2.567 1.41s.524-.511 1.479-.883 1.861-.191 1.861-.191-.598.54-1.615.935c-1.016.397-1.725.139-1.725.139m-2.493.505s.545-.224 1.357-.196 1.415.47 1.415.47-.608.222-1.473.193-1.3-.467-1.3-.467m6.186-4.203s.175-1.008.974-2.154c.8-1.147 2.015-1.578 2.015-1.578s-.238 1.098-1.089 2.319c-.85 1.22-1.9 1.413-1.9 1.413m1.003-3.071s-.195-1.021.133-2.393c.33-1.371 1.293-2.21 1.293-2.21s.17 1.128-.18 2.589c-.349 1.46-1.246 2.014-1.246 2.014M12 20.047a.413.413 0 1 0 0-.827.413.413 0 0 0 0 .827"></path></svg></div>Travellers’ Choice</div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Preço</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r26a:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26a:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="priceTypes.10953" aria-labelledby="priceTypes.10953_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="priceTypes.10953" value="10953"></div><div class="hTbJa f H o"><label id="priceTypes.10953_label" for="priceTypes.10953" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Comer barato</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="priceTypes.10955" aria-labelledby="priceTypes.10955_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="priceTypes.10955" value="10955"></div><div class="hTbJa f H o"><label id="priceTypes.10955_label" for="priceTypes.10955" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Intermediário</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="priceTypes.10954" aria-labelledby="priceTypes.10954_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="priceTypes.10954" value="10954"></div><div class="hTbJa f H o"><label id="priceTypes.10954_label" for="priceTypes.10954" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Restaurante sofisticado</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Pontuação</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r26e:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26e:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="minimumTravelerRating.TRAVELER_RATING_LOW" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_LOW_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_LOW" value="TRAVELER_RATING_LOW"></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_LOW_label" for="minimumTravelerRating.TRAVELER_RATING_LOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-r26g:" data-automation="bubbleRatingImage"><title id=":lithium-r26g:">3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P pZUbB biKBZ AWdfh">ou mais</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="minimumTravelerRating.TRAVELER_RATING_MEDIUM" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_MEDIUM" value="TRAVELER_RATING_MEDIUM"></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" for="minimumTravelerRating.TRAVELER_RATING_MEDIUM" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-r26i:" data-automation="bubbleRatingImage"><title id=":lithium-r26i:">4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P pZUbB biKBZ AWdfh">ou mais</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="minimumTravelerRating.TRAVELER_RATING_HIGH" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_HIGH_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_HIGH" value="TRAVELER_RATING_HIGH"></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_HIGH_label" for="minimumTravelerRating.TRAVELER_RATING_HIGH" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-r26k:" data-automation="bubbleRatingImage"><title id=":lithium-r26k:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg></div></div></div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Aberto agora</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r26l:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26l:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="openNow.OPEN_NOW" aria-labelledby="openNow.OPEN_NOW_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="openNow.OPEN_NOW" value="OPEN_NOW"></div><div class="hTbJa f H o"><label id="openNow.OPEN_NOW_label" for="openNow.OPEN_NOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Aberto agora</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Restrições alimentares</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r26n:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26n:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diets.10665" aria-labelledby="diets.10665_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diets.10665" value="10665"></div><div class="hTbJa f H o"><label id="diets.10665_label" for="diets.10665" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Adequado para vegetarianos</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diets.10697" aria-labelledby="diets.10697_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diets.10697" value="10697"></div><div class="hTbJa f H o"><label id="diets.10697_label" for="diets.10697" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Opções veganas</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diets.10992" aria-labelledby="diets.10992_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diets.10992" value="10992"></div><div class="hTbJa f H o"><label id="diets.10992_label" for="diets.10992" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Opções sem glúten</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Excelente para</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r26r:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26r:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10604" aria-labelledby="styles.10604_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10604" value="10604"></div><div class="hTbJa f H o"><label id="styles.10604_label" for="styles.10604" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Famílias com crianças</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10609" aria-labelledby="styles.10609_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10609" value="10609"></div><div class="hTbJa f H o"><label id="styles.10609_label" for="styles.10609" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Grupos grandes</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10605" aria-labelledby="styles.10605_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10605" value="10605"></div><div class="hTbJa f H o"><label id="styles.10605_label" for="styles.10605" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Reunião de negócios</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10608" aria-labelledby="styles.10608_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10608" value="10608"></div><div class="hTbJa f H o"><label id="styles.10608_label" for="styles.10608" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Atmosfera</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button" aria-expanded="false" aria-controls="filter-expando-styles"><span class="biGQs _P ezezH">Mostrar mais<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Características</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r270:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r270:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.16547" aria-labelledby="diningOptions.16547_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.16547" value="16547"></div><div class="hTbJa f H o"><label id="diningOptions.16547_label" for="diningOptions.16547" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Serviço de mesa</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.10852" aria-labelledby="diningOptions.10852_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.10852" value="10852"></div><div class="hTbJa f H o"><label id="diningOptions.10852_label" for="diningOptions.10852" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Lugares para sentar</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.10601" aria-labelledby="diningOptions.10601_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.10601" value="10601"></div><div class="hTbJa f H o"><label id="diningOptions.10601_label" for="diningOptions.10601" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Para levar</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.10861" aria-labelledby="diningOptions.10861_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.10861" value="10861"></div><div class="hTbJa f H o"><label id="diningOptions.10861_label" for="diningOptions.10861" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Acesso para cadeirantes</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button"><span class="biGQs _P ezezH">Mostrar tudo</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Bairro</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r275:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r275:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945488" aria-labelledby="neighbourhoods.20945488_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945488" value="20945488"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945488_label" for="neighbourhoods.20945488" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Umarizal</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945487" aria-labelledby="neighbourhoods.20945487_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945487" value="20945487"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945487_label" for="neighbourhoods.20945487" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Nazare</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945485" aria-labelledby="neighbourhoods.20945485_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945485" value="20945485"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945485_label" for="neighbourhoods.20945485" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Campina</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945486" aria-labelledby="neighbourhoods.20945486_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945486" value="20945486"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945486_label" for="neighbourhoods.20945486" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Fatima</span></span></span></label></div></span></div></div></div></div></div></span></div></div><div class="oqGNy" data-automation="LeftRailSlot"><div class="uGSLz f e"><div class="IUxgK"><div class="CsVUk E" style="top: 24px;"><div class="YcOBw mFLzV gGJkl"><div class="Eufjb j u" id="slot:300x600:rail1" data-google-query-id="CILs0Kzq2I4DFbY5uQYd5NY64A"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_38__container__" style="border: 0pt none; display: inline-block; width: 300px; height: 600px;"><iframe frameborder="0" src="https://f317dcf7d6522885a606297e1d4c070c.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_38" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="300" height="600" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Advertisement" tabindex="0" data-google-container-id="a" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></div></div></div></div><div class="IUxgK"><div class="CsVUk E" style="top: 24px;"><div class="YcOBw mFLzV gGJkl"><div class="Eufjb j u ChFsW Xd o S" id="slot:300x600:rail3"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_39__container__" style="border: 0pt none; display: inline-block;"></div></div></div></div></div></div></div></div><div class="LvzMf _T" data-automation="LeftRailMain"><div class="u f Q2"><div class="biGQs _P pZUbB AWdfh" data-automation="resultsTotal">1.096 resultados</div><div class="NB"><span><div class="f u"><div class="biGQs _P pZUbB egaXP kSNRl ZNjnF">Ordenar por: </div><div class="lOvTR"><button class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" type="button"><span class="biGQs _P ezezH">Em destaque</span><div class="YQrIS _W PH"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div><div class="DgJpu"><button type="button" class="ypcsE _S bYExr" aria-labelledby=":lithium-r27b:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb rihVH UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-r27b:" class="KrpDA o">Restaurantes classificados de acordo com visualizações de página, avaliações e atributos individuais, como faixa de preço, culinária e localização, bem como dados agregados do Tripadvisor que comparam as interações dos usuários com todas as outras páginas de restaurantes neste local.</span></div></div></span></div></div><div class="fKQIr"><div class="rfqBq f e"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">1. Restaurante Ver O Açaí - Umarizal</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r27i:" data-automation="bubbleRatingImage"><title id=":lithium-r27i:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Bar</span></span><span class="biGQs _P pZUbB ZNjnF"></span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://veroacai.com.br/lp/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sensacional</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale a experiencia </span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-23588946"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">2. Amazônia Na Cuia</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r27p:" data-automation="bubbleRatingImage"><title id=":lithium-r27p:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(92 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://pedidosup.com.br/amazonianacuiadelivery/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Belém</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Muito bom</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="Campeão de venda chapa mista paraense com açaí." loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="pirarucu com açaí" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="Fotos pratos do Point" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6483240"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">3. Restaurante Point do Açaí</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r280:" data-automation="bubbleRatingImage"><title id=":lithium-r280:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(4.884 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc bYExr jWkoZ QHaGY VauJM" href="/UserReviewEdit-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div><div class=""><div id="slot:728x90:inline3" class="Eufjb j u nFcpI" data-google-query-id="CPKbzqzq2I4DFbY5uQYd5NY64A"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_40__container__" style="border: 0pt none; display: inline-block; width: 100%; height: auto;"><iframe frameborder="0" src="https://f317dcf7d6522885a606297e1d4c070c.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_40" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="0" height="190" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Advertisement" tabindex="0" data-google-container-id="c" style="border: 0px; vertical-align: bottom; min-width: 100%;" data-load-complete="true"></iframe></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373246"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">4. Manjar das Garças</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r287:" data-automation="bubbleRatingImage"><title id=":lithium-r287:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.206 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa comida, mas com preço elevado</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente programa.</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2360313"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">5. Remanso do Peixe</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r28e:" data-automation="bubbleRatingImage"><title id=":lithium-r28e:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(915 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Saboroso</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Remanseando no Remanso!</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-19450192"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">6. Casa Do Saulo Das Onze Janelas</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r28l:" data-automation="bubbleRatingImage"><title id=":lithium-r28l:">4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(166 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Contemporânea</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="http://linklist.bio/casadosaulo" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale a visita apesar de.....</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale sim a visita </span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25365698"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">7. Santa Grelha Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r28s:" data-automation="bubbleRatingImage"><title id=":lithium-r28s:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(182 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Steakhouse</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente lugar</span></div></a></div></div></div></div></div></div><div class="LthNE qWPrE ncFvv" data-automation="shelf"><hr class="RMdFe tyUdl"><span><div class="vqnnc"><div class="hHadL f k Q3 u MJ"><div class="biGQs _P fiohW KeZJf">Restaurantes sofisticados</div><a class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" href="/Restaurants-g303404-zfp10954-Belem_State_of_Para.html" data-automation="seeAllLink" target="_blank"><span class="biGQs _P ezezH">Veja tudo</span></a></div><div class="aEwIm"><div class="YRcYv f Q3 WDFZm _h"><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="323" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Umarizal</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r293:" data-automation="bubbleRatingImage"><title id=":lithium-r293:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25377098"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Ryori belém</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29a:" data-automation="bubbleRatingImage"><title id=":lithium-r29a:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/27/a9/3d/9e/sempre-renovando-nossa.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373223"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Famiglia Sicilia</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29h:" data-automation="bubbleRatingImage"><title id=":lithium-r29h:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(451 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Italiana • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="301" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373246"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Manjar das Garcas</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29o:" data-automation="bubbleRatingImage"><title id=":lithium-r29o:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.206 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0b/8d/32/ad/images-1-largejpg.jpg?w=300&amp;h=-1&amp;s=1" width="300" height="250" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373270"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Grand Cru</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29v:" data-automation="bubbleRatingImage"><title id=":lithium-r29v:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(463 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Europeia • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6009425"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sushi Ruy Barbosa</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2a6:" data-automation="bubbleRatingImage"><title id=":lithium-r2a6:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d6009066-Reviews-Santa_Chicoria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6009066"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Santa Chicoria</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ad:" data-automation="bubbleRatingImage"><title id=":lithium-r2ad:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d6009066-Reviews-Santa_Chicoria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(188 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d25846365-Reviews-Zu_Gourmande-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25846365"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Zu Gourmande</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ak:" data-automation="bubbleRatingImage"><title id=":lithium-r2ak:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d25846365-Reviews-Zu_Gourmande-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(22 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Francesa • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d23445371-Reviews-Purao_Vegano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-23445371"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Purão Vegano</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ar:" data-automation="bubbleRatingImage"><title id=":lithium-r2ar:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d23445371-Reviews-Purao_Vegano-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(11 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d3507098-Reviews-A_Forneria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3507098"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">A Forneria</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>3,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2b2:" data-automation="bubbleRatingImage"><title id=":lithium-r2b2:">3,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d3507098-Reviews-A_Forneria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(136 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Italiana • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div></div><div class="zOqpq u f S t w"><div class="nxlXU S CxuXo"></div><div class="iWPJm u k l _Q w Z1"><div class="eaUlz H"></div><div class="HFJew"><button class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" type="button" aria-label="Avançar"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div></div></div></div></div></span><hr class="RMdFe roAGK"></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25377098"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">8. Ryori belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2b9:" data-automation="bubbleRatingImage"><title id=":lithium-r2b9:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa experiência de japonês</span></div></a></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline4" class="Eufjb j u ChFsW Xd o S nFcpI"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_41__container__" style="border: 0pt none; display: inline-block;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-27765971"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">9. Restaurante Ver O Açaí - Marco</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2bg:" data-automation="bubbleRatingImage"><title id=":lithium-r2bg:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(339 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://veroacai.com.br/lp/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Comida e atendimento por excelência.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Experimente a Amazônia em um lugar</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24040497"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">10. Camarada Camarão Belém - Shopping Boulevard Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2bn:" data-automation="bubbleRatingImage"><title id=":lithium-r2bn:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(1.698 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://livemenu.app/menu/627bcbac218bd50011827877" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Camarada.</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-13437655"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">11. Geek Burger</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2bu:" data-automation="bubbleRatingImage"><title id=":lithium-r2bu:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(375 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Americana, Bar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://linktr.ee/geekburgeroficial" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Divertido</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3849186"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">12. Govinda Restaurante Vegetariano</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,7</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2c5:" data-automation="bubbleRatingImage"><title id=":lithium-r2c5:">4,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(303 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Indiana, Saudável</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://micrositios.ifood.com.br/delivery/belem-pa/govinda-restaurante-vegetariano-batista-campos/56e69336-66ec-42f7-ab96-43d70c523be1" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Não sou vegana, mas amo a experiência gastronômica.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Govinda vegetariano Belém</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=700&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3259642"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">13. Saldosa Maloca</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2cc:" data-automation="bubbleRatingImage"><title id=":lithium-r2cc:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(316 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Experiência.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sabores do Pará.</span></div></a></div></div></div></div></div></div><div class="LthNE qWPrE ncFvv" data-automation="shelf"><hr class="RMdFe tyUdl"><span><div class="vqnnc"><div class="hHadL f k Q3 u MJ"><div class="biGQs _P fiohW KeZJf">Restaurantes locais</div><a class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" href="/Restaurants-g303404-zft10613-Belem_State_of_Para.html" data-automation="seeAllLink" target="_blank"><span class="biGQs _P ezezH">Veja tudo</span></a></div><div class="aEwIm"><div class="YRcYv f Q3 WDFZm _h"><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="323" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Umarizal</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2cj:" data-automation="bubbleRatingImage"><title id=":lithium-r2cj:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6483240"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Point do Açai</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2cq:" data-automation="bubbleRatingImage"><title id=":lithium-r2cq:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(4.884 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-26839232"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Ver-o-Pesinho</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2d1:" data-automation="bubbleRatingImage"><title id=":lithium-r2d1:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(299 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-27765971"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Marco</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2d8:" data-automation="bubbleRatingImage"><title id=":lithium-r2d8:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(339 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2360313"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Remanso do Peixe</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2df:" data-automation="bubbleRatingImage"><title id=":lithium-r2df:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(915 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="301" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373246"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Manjar das Garcas</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2dm:" data-automation="bubbleRatingImage"><title id=":lithium-r2dm:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.206 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3902846"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Cervejaria Amazon Beer</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2dt:" data-automation="bubbleRatingImage"><title id=":lithium-r2dt:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.488 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3259642"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Saldosa Maloca</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2e4:" data-automation="bubbleRatingImage"><title id=":lithium-r2e4:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(316 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-12895755"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Point do Acai</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,7</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2eb:" data-automation="bubbleRatingImage"><title id=":lithium-r2eb:">4,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(159 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373303"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Tacaca da Dona Maria</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ei:" data-automation="bubbleRatingImage"><title id=":lithium-r2ei:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(287 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $</div></div></div></a></span></div></div><div class="zOqpq u f S t w"><div class="nxlXU S CxuXo"></div><div class="iWPJm u k l _Q w Z1"><div class="eaUlz H"></div><div class="HFJew"><button class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" type="button" aria-label="Avançar"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div></div></div></div></div></span><hr class="RMdFe roAGK"></div><div class=""><div id="slot:728x90:inline5" class="Eufjb j u ChFsW Xd o S nFcpI"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_42__container__" style="border: 0pt none; display: inline-block;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-19246676"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">14. Sushi Boulevard</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ep:" data-automation="bubbleRatingImage"><title id=":lithium-r2ep:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(497 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Sushi</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc bYExr jWkoZ QHaGY VauJM" href="/UserReviewEdit-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-12895755"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">15. Point do Açaí</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,7</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2f0:" data-automation="bubbleRatingImage"><title id=":lithium-r2f0:">4,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(159 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Visita gastronômica</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Luciano: garçom incrível.</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3902846"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">16. Cervejaria Amazon Beer</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2f7:" data-automation="bubbleRatingImage"><title id=":lithium-r2f7:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.488 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Pub com cerveja artesanal</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://pedidosup.com.br/amazonbeercardapio/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Cervejar e petiscar</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Chopps especiais</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=700&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-10158580"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">17. Sushi Boulevard</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2fe:" data-automation="bubbleRatingImage"><title id=":lithium-r2fe:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(790 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Sushi</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="http://www.sushiboulevard.com.br/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc bYExr jWkoZ QHaGY VauJM" href="/UserReviewEdit-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/12/f5/ac/52/ta-img-20180515-201010.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/12/f5/ac/52/ta-img-20180515-201010.jpg?w=700&amp;h=700&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/12/f5/ac/52/ta-img-20180515-201010.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-4574569"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">18. Xicara da Silva</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2fl:" data-automation="bubbleRatingImage"><title id=":lithium-r2fl:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(539 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Pizza</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Jantar</span></div></a></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline6" class="Eufjb j u ChFsW Xd o S nFcpI"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_43__container__" style="border: 0pt none; display: inline-block;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-857993"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">19. Roxy Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2fs:" data-automation="bubbleRatingImage"><title id=":lithium-r2fs:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(1.584 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vá que é bom.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Tudo muito bom! Recomendamos! </span></div></a></div></div></div></div></div></div><div class="LthNE qWPrE ncFvv" data-automation="shelf"><hr class="RMdFe tyUdl"><span><div class="vqnnc"><div class="hHadL f k Q3 u MJ"><div class="biGQs _P fiohW KeZJf">Restaurantes acessíveis</div><a class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" href="/Restaurants-g303404-zfp10955-Belem_State_of_Para.html" data-automation="seeAllLink" target="_blank"><span class="biGQs _P ezezH">Veja tudo</span></a></div><div class="aEwIm"><div class="YRcYv f Q3 WDFZm _h"><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="323" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Umarizal</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2g3:" data-automation="bubbleRatingImage"><title id=":lithium-r2g3:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24040497"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Camarada Camarão Belém - Shopping Boulevard Belém</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ga:" data-automation="bubbleRatingImage"><title id=":lithium-r2ga:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(1.698 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-26839232"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Ver-o-Pesinho</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2gh:" data-automation="bubbleRatingImage"><title id=":lithium-r2gh:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(299 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-27765971"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Marco</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2go:" data-automation="bubbleRatingImage"><title id=":lithium-r2go:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(339 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24995218"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Coco Bambu Belém</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2gv:" data-automation="bubbleRatingImage"><title id=":lithium-r2gv:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(950 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-13437655"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Geek Burger</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2h6:" data-automation="bubbleRatingImage"><title id=":lithium-r2h6:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(375 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Americana • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25365698"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">SANTA GRELHA BELÉM</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2hd:" data-automation="bubbleRatingImage"><title id=":lithium-r2hd:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(182 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Steakhouse • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-19246676"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sushi Boulevard</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2hk:" data-automation="bubbleRatingImage"><title id=":lithium-r2hk:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(497 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-10158580"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sushi Boulevard</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2hr:" data-automation="bubbleRatingImage"><title id=":lithium-r2hr:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(790 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d7747107-Reviews-Sorveteria_Cairu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-7747107"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sorveteria Cairu</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2i2:" data-automation="bubbleRatingImage"><title id=":lithium-r2i2:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d7747107-Reviews-Sorveteria_Cairu-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.250 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div></div></div></a></span></div></div><div class="zOqpq u f S t w"><div class="nxlXU S CxuXo"></div><div class="iWPJm u k l _Q w Z1"><div class="eaUlz H"></div><div class="HFJew"><button class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" type="button" aria-label="Avançar"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div></div></div></div></div></span><hr class="RMdFe roAGK"></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24995218"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">20. Coco Bambu Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2i9:" data-automation="bubbleRatingImage"><title id=":lithium-r2i9:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(950 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Café</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Agregou mais valor a nossa viagem 😍</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Comida e atendimento excelentes</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-23315259"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">21. Tap House Boteco Ver-a-Cerva</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ig:" data-automation="bubbleRatingImage"><title id=":lithium-r2ig:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(60 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Bares e pubs, Americana</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Cervejas e Drinks em Belém</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sensacional</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/27/a9/3d/9e/sempre-renovando-nossa.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373223"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">22. Famiglia Sicilia</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2in:" data-automation="bubbleRatingImage"><title id=":lithium-r2in:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(451 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Italiana, Europeia</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Uma experiência surpreendente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Italiano paraense de respeito</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25432845"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">23. Tap House Ver-A-Cerva - Batista Campos</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2iu:" data-automation="bubbleRatingImage"><title id=":lithium-r2iu:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(38 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Bares e pubs, Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://veracerva.com.br/lp/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Otimo local para tomar cerveja e ouvir excelentes músicas, con atendimento muito atencioso</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Gostoso!</span></div></a></div></div></div></div></div></div><div data-automation="nova-ad-InlineNova"><div class="VnzpY"><div id="slot:728x90-3x3:InlineNova" class="RnYjA j u ZWKar Xd y o"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_44__container__" style="border: 0pt none; display: inline-block;"></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-8624369"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">24. Porpino Burger</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,6</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2j5:" data-automation="bubbleRatingImage"><title id=":lithium-r2j5:">4,6 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(833 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Americana, Bar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://porpinoburger.com/menu/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Maravilhoso!</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-8826462"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">25. restaurante chalé da Ilha do combu.</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,2</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2jc:" data-automation="bubbleRatingImage"><title id=":lithium-r2jc:">4,2 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(118 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Bar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Estrutura magnífica, serviço lento</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Um dos melhores bares/restaurantes da Ilha</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6009425"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">26. Sushi Ruy Barbosa</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2jj:" data-automation="bubbleRatingImage"><title id=":lithium-r2jj:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Sushi</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Muito bom, mas caro.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Ambiente muito agradável</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373303"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">27. Tacacá da Dona Maria</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2jq:" data-automation="bubbleRatingImage"><title id=":lithium-r2jq:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(287 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$</span></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">No centro de Belém</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">delicioso</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3193396"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">28. La Traviata</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2k1:" data-automation="bubbleRatingImage"><title id=":lithium-r2k1:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(532 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Italiana, Pizza</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Dicas para o proprietario</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">O forte é a comida italiana</span></div></a></div></div></div></div></div></div><div class=""><div class="NxBlo Na"><a target="_blank" href="https://www.tripadvisor.com.br/Articles" class="BMQDV _F Gv bYExr SwZTJ hNpWR"><section aria-label="Continue planejando O que fazer, onde comer e mais inspirações de viagem. Veja mais" class="BQTWL f zQRfF"><div class="Bsnup f"><div class="dsJsk o"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/2d/40/95/keep-on-planning.jpg?w=200&amp;h=200&amp;s=1&amp;cx=492&amp;cy=358&amp;chk=v1_f64df95338d294710736 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/2d/40/95/keep-on-planning.jpg?w=400&amp;h=400&amp;s=1&amp;cx=492&amp;cy=358&amp;chk=v1_f64df95338d294710736 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/2d/40/95/keep-on-planning.jpg?w=200&amp;h=200&amp;s=1&amp;cx=492&amp;cy=358&amp;chk=v1_f64df95338d294710736" width="200" height="200" alt="" loading="lazy"></picture></div></div><div class="GiBeo o F4 f"><div class="LVEbf f"><div class=""><div class="biGQs _P fiohW alXOW EEXWj GzNcM BYtua UTQMg alvrA KeZJf">Continue planejando</div></div><div class=""><div class="biGQs _P fiohW alXOW EEXWj GzNcM BYtua UTQMg alvrA AWdfh">O que fazer, onde comer e mais inspirações de viagem.</div></div></div><div class="mJfQs f"><div class="CwwAT"><button class="rmyCe _G B- z _S c Wc bYExr w AeLHi QHaGY VauJM" type="button"><span class="biGQs _P ezezH">Veja mais</span></button></div></div></div></section></a></div><div id="slot:728x90:inline7" class="Eufjb j u ChFsW Xd o S nFcpI" data-google-query-id="CPebzqzq2I4DFbY5uQYd5NY64A" style="display: none;"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_45__container__" style="border: 0pt none; width: 728px; height: 0px;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0b/8d/32/ad/images-1-largejpg.jpg?w=300&amp;h=300&amp;s=1" width="300" height="250" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373270"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">29. Grand Cru</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2k8:" data-automation="bubbleRatingImage"><title id=":lithium-r2k8:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(463 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Bares e pubs, Europeia</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Ambiente muito agradável</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vinhos excitantes</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-15273271"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">30. UATA?!</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2kf:" data-automation="bubbleRatingImage"><title id=":lithium-r2kf:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(144 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Fast food</span></span><span class="biGQs _P pZUbB ZNjnF">$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Ótimos, sorridentes e super competentes os atendimentos</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amei o atendimento</span></div></a></div></div></div></div></div></div><div class="qAZoU c"><div class="GMYGA"><div class="mkNRT j"><div class="HXmEd"><div class="IGLCo"><a class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" data-smoke-attr="pagination-next-arrow" aria-label="Próxima página" href="/Restaurants-g303404-oa30-Belem_State_of_Para.html"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></a></div></div></div><div class="LaMdn j"><div class="aMnVW"><div class="Yzhnw P"><button class="BrOJk u j z _F _S bYExr tIqAi iNBVo SSqtP xtNBb" type="button" disabled="" aria-label="1"><span class="kLqdM"><span class="biGQs _P ezezH">1</span></span></button></div><div class="Yzhnw P"><a class="BrOJk u j z _F _S bYExr tIqAi iNBVo xtNBb" aria-label="2" href="/Restaurants-g303404-oa30-Belem_State_of_Para.html"><span class="kLqdM"><span class="biGQs _P ezezH">2</span></span></a></div><div class="Yzhnw P"><div class="biGQs _P fiohW ezezH"><div class="sUgyD">…</div></div></div></div></div></div></div><div class="qAZoU c"><div class="biGQs _P pZUbB ZNjnF"><div class="Ci">Mostrando 1–30 de 1.096 resultados</div></div></div></div></div></div></div></div></div>"""

# Criando um objeto BeautifulSoup a partir do HTML completo
soup_completo = BeautifulSoup(html_completo_exemplos, 'html.parser')

# Encontrar todos os blocos de restaurante
# A classe 'tbrcR' combinada com outras parece ser um bom seletor para cada "cartão" de restaurante.
restaurant_blocks = soup_completo.find_all('div', class_='tbrcR')

print("Iniciando a extração detalhada de cada restaurante:")
restaurantes_detalhados = [] # Lista para armazenar os dados estruturados

for block in restaurant_blocks:
    # --- Seletores para as informações DENTRO de cada bloco de restaurante ---

    # 1. Nome do Restaurante
    # Pistas: O nome está dentro de uma <a> que tem as classes 'BMQDV' e 'ukgoS'.
    # O texto real do nome está dentro de uma <div> que é filha dessa <a>.
    # Exemplo HTML: <a class="BMQDV ... ukgoS"><div class="biGQs ...">1. Nome do Restaurante</div></a>
    nome_link_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'ukgoS' in c)
    nome = "Nome não encontrado"
    if nome_link_tag:
        # A div específica que contém o texto do nome
        nome_div_tag = nome_link_tag.find('div', class_=lambda c: c and 'biGQs' in c and 'fiohW' in c)
        if nome_div_tag:
            nome = nome_div_tag.get_text(strip=True)

    # 2. Avaliação (e.g., 4,9)
    # Pistas: Encontrar a div com data-automation="bubbleRatingValue" e pegar o texto do span dentro.
    avaliacao_tag = block.find('div', {'data-automation': 'bubbleRatingValue'})
    avaliacao = avaliacao_tag.find('span').get_text(strip=True) if avaliacao_tag else "Avaliação não encontrada"

    # 3. Contagem de Reviews
    # Pistas: Está em uma <div> com o atributo 'data-automation="bubbleReviewCount"'.
    # O texto real da contagem está dentro de um <span> que é filho dessa <div>.
    # Usamos regex para extrair apenas os números.
    review_count_tag = block.find('div', {'data-automation': 'bubbleReviewCount'})
    contagem_reviews = "Contagem de reviews não encontrada"
    if review_count_tag:
        text_content = review_count_tag.get_text(strip=True)
        # Extrair apenas os números da string (ex: "(2.529 avaliações)" -> "2.529")
        match = re.search(r'\d[\d.]*', text_content)
        if match:
            contagem_reviews = match.group(0).replace('.', '') # Remove o ponto para converter para int, se necessário

    # 4. Tipo de Cozinha (ex: Brasileira, Bar)
    # Pistas: É um <span> dentro de um <div> com classes 'ZvrsW N G biqBm'.
    # O texto pode estar diretamente no <span> ou em um <span> aninhado.
    # Tentaremos pegar o primeiro span que não contém um SVG ou um link.
    cuisine_tag = block.find('div', class_=lambda c: c and 'ZvrsW' in c and 'biqBm' in c)
    tipo_cozinha = []
    if cuisine_tag:
        # Encontrar todos os spans que são filhos diretos ou quase diretos e que contêm texto relevante
        spans = cuisine_tag.find_all('span', class_=lambda c: c and 'biGQs' in c and 'pZUbB' in c)
        for span in spans:
            # Excluir spans que contêm SVGs (ícones) ou que parecem ser contadores de reviews
            # e pegar apenas o texto que não é de contagem de reviews ou "Fechado agora" ou "Cardápio"
            if not span.find('svg') and 'avaliações' not in span.get_text() and 'Fechado agora' not in span.get_text() and 'Cardápio' not in span.get_text():
                cuisine_text = span.get_text(strip=True)
                if cuisine_text:
                    tipo_cozinha.append(cuisine_text)
    tipo_cozinha_str = ", ".join(tipo_cozinha) if tipo_cozinha else "Tipo de cozinha não encontrado"

    # 5. Faixa de Preço (ex: $$ - $$$)
    # Pistas: Está em um <span> com as mesmas classes do tipo de cozinha, mas é o que não tem SVG
    # e que corresponde ao padrão de preço.
    faixa_preco = "Faixa de preço não encontrada"
    if cuisine_tag:
        for span in cuisine_tag.find_all('span', class_=lambda c: c and 'biGQs' in c and 'pZUbB' in c):
            text_content = span.get_text(strip=True)
            if re.match(r'\$+', text_content): # Verifica se o texto é um ou mais símbolos de dólar
                faixa_preco = text_content
                break

    # 6. Status de Funcionamento (Aberto/Fechado)
    # Pistas: Está dentro de uma div com classe 'Pwlnc f RAyMq', e o texto real no span dentro.
    status_funcionamento_tag = block.find('div', class_=lambda c: c and 'Pwlnc' in c and 'RAyMq' in c)
    status_funcionamento = status_funcionamento_tag.find('span').get_text(strip=True) if status_funcionamento_tag else "Status de funcionamento não encontrado"

    # 7. URL do Restaurante
    # Pistas: A principal tag <a> que envolve todo o bloco do restaurante.
    # O atributo 'href' contém o caminho relativo. Precisamos concatenar com a base do Tripadvisor.
    url_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'PaRlG' in c)
    url_base = "https://www.tripadvisor.com.br"
    url_restaurante = url_base + url_tag['href'] if url_tag and 'href' in url_tag.attrs else "URL não encontrada"

    # 8. URL da Imagem Principal
    # Pistas: A primeira tag <picture> dentro do carousel, e a tag <img> dentro dela.
    # O atributo 'src' contém a URL da imagem.
    image_tag = block.find('div', class_='IdURT').find('img') if block.find('div', class_='IdURT') else None
    url_imagem = image_tag['src'] if image_tag and 'src' in image_tag.attrs else "URL da imagem não encontrada"

    # Armazenar os dados em um dicionário
    restaurante_info = {
        "Nome": nome,
        "Avaliacao": avaliacao,
        "Contagem_Reviews": contagem_reviews,
        "Tipo_Cozinha": tipo_cozinha_str,
        "Faixa_Preco": faixa_preco,
        "Status_Funcionamento": status_funcionamento,
        "URL_Restaurante": url_restaurante,
        "URL_Imagem_Principal": url_imagem
    }
    restaurantes_detalhados.append(restaurante_info)

# Imprimir os resultados para verificação
for i, restaurante in enumerate(restaurantes_detalhados):
    print(f"\n--- Restaurante {i+1} ---")
    for key, value in restaurante.items():
        print(f"{key}: {value}")

Iniciando a extração detalhada de cada restaurante:

--- Restaurante 1 ---
Nome: 1. Restaurante Ver O Açaí - Umarizal
Avaliacao: 4,9
Contagem_Reviews: 2529
Tipo_Cozinha: Brasileira, Bar
Faixa_Preco: Faixa de preço não encontrada
Status_Funcionamento: Fechado agora
URL_Restaurante: https://www.tripadvisor.com.br/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html
URL_Imagem_Principal: https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&h=400&s=1

--- Restaurante 2 ---
Nome: 2. Amazônia Na Cuia
Avaliacao: 4,4
Contagem_Reviews: 92
Tipo_Cozinha: Brasileira, Frutos do mar, $$ - $$$
Faixa_Preco: $$ - $$$
Status_Funcionamento: Status de funcionamento não encontrado
URL_Restaurante: https://www.tripadvisor.com.br/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html
URL_Imagem_Principal: https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-p

In [12]:
from bs4 import BeautifulSoup

html_do_tripadvisor = """
<<div class="IDaDx Iwmxp mvTrV cyIij fluiI SMjpI"><div class="u f k mowmC"><div class="USSyL" data-test-target="main_h1"><h2 class="biGQs _P fiohW eTlmC">Restaurantes: Belém</h2></div></div><div class="u f k tyUdl"><div class="unvTU"><span class="biGQs _P fiohW RnSgX">Melhores restaurantes em Belém</span><div class="gsRoG NK _Q"><div class="DgJpu"><button type="button" class="ypcsE _S bYExr" aria-labelledby=":lithium-r25i:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-r25i:" class="KrpDA o">Restaurantes classificados de acordo com visualizações de página, avaliações e atributos individuais, como faixa de preço, culinária e localização, bem como dados agregados do Tripadvisor que comparam as interações dos usuários com todas as outras páginas de restaurantes neste local.</span></div></div></div><div class="Mjzwz u f K"><span data-automation="mapButton"><button class="EAeYu z Xa q Pc PQ Pp PD W _S Gy Rd Cr _M pGzPy bYExr ncKoe" type="button" aria-label="Ver mapa"><span class="sZLBj _Y k Q1 ncKoe"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg><span class="biGQs _P AWdfh">Mapa</span></span></button></span></div></div><div class="rxvcc"><div class="seiBA _T wCxWg" data-automation="LeftRailContainer"><div class="wjLQL _T Ft" data-automation="LeftRail"><div class="vILDq e fbkSn"><div class="WVPPt S5 Gi R4 f e EUDqa"><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Tipo</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r25j:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r25j:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.10591" aria-labelledby="establishmentTypes.10591_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="true" name="establishmentTypes.10591" value="10591" checked=""></div><div class="hTbJa f H o"><label id="establishmentTypes.10591_label" for="establishmentTypes.10591" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Restaurantes</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.9909" aria-labelledby="establishmentTypes.9909_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="establishmentTypes.9909" value="9909"></div><div class="hTbJa f H o"><label id="establishmentTypes.9909_label" for="establishmentTypes.9909" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Docerias</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.9900" aria-labelledby="establishmentTypes.9900_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="establishmentTypes.9900" value="9900"></div><div class="hTbJa f H o"><label id="establishmentTypes.9900_label" for="establishmentTypes.9900" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Café e chá</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="establishmentTypes.11776" aria-labelledby="establishmentTypes.11776_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="establishmentTypes.11776" value="11776"></div><div class="hTbJa f H o"><label id="establishmentTypes.11776_label" for="establishmentTypes.11776" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Bares e pubs</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button" aria-expanded="false" aria-controls="filter-expando-establishmentTypes"><span class="biGQs _P ezezH">Mostrar mais<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Tipo de refeição</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r25o:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r25o:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10597" aria-labelledby="mealTypes.10597_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10597" value="10597"></div><div class="hTbJa f H o"><label id="mealTypes.10597_label" for="mealTypes.10597" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Café da manhã</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10606" aria-labelledby="mealTypes.10606_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10606" value="10606"></div><div class="hTbJa f H o"><label id="mealTypes.10606_label" for="mealTypes.10606" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Brunch</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10598" aria-labelledby="mealTypes.10598_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10598" value="10598"></div><div class="hTbJa f H o"><label id="mealTypes.10598_label" for="mealTypes.10598" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Almoço</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="mealTypes.10599" aria-labelledby="mealTypes.10599_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="mealTypes.10599" value="10599"></div><div class="hTbJa f H o"><label id="mealTypes.10599_label" for="mealTypes.10599" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Jantar</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Culinárias</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r25t:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r25t:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10348" aria-labelledby="cuisines.10348_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10348" value="10348"></div><div class="hTbJa f H o"><label id="cuisines.10348_label" for="cuisines.10348" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Brasileira</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10749" aria-labelledby="cuisines.10749_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10749" value="10749"></div><div class="hTbJa f H o"><label id="cuisines.10749_label" for="cuisines.10749" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Sul-americana</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10641" aria-labelledby="cuisines.10641_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10641" value="10641"></div><div class="hTbJa f H o"><label id="cuisines.10641_label" for="cuisines.10641" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Pizza</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="cuisines.10670" aria-labelledby="cuisines.10670_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="cuisines.10670" value="10670"></div><div class="hTbJa f H o"><label id="cuisines.10670_label" for="cuisines.10670" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Pub</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button"><span class="biGQs _P ezezH">Mostrar tudo</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Pratos</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r262:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r262:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.23001" aria-labelledby="dishes.23001_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.23001" value="23001"></div><div class="hTbJa f H o"><label id="dishes.23001_label" for="dishes.23001" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Camarão VG</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.10647" aria-labelledby="dishes.10647_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.10647" value="10647"></div><div class="hTbJa f H o"><label id="dishes.10647_label" for="dishes.10647" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Lanches</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.10893" aria-labelledby="dishes.10893_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.10893" value="10893"></div><div class="hTbJa f H o"><label id="dishes.10893_label" for="dishes.10893" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Caranguejo</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="dishes.20336" aria-labelledby="dishes.20336_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="dishes.20336" value="20336"></div><div class="hTbJa f H o"><label id="dishes.20336_label" for="dishes.20336" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Bife com batata frita</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button"><span class="biGQs _P ezezH">Mostrar tudo</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH"><div class="f">Prêmios<div class="TtrFj"><div class="biGQs _P ezezH"><span class="NK"><div class="DgJpu"><button type="button" class="ypcsE _S bYExr" aria-labelledby=":lithium-r268:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-r268:" class="KrpDA o">Os vencedores do Prêmio Travellers’ Choice (incluindo o título “Os melhores dos melhores”) estão entre os 10% dos melhores perfis no Tripadvisor, de acordo com avaliações e opiniões de viajantes do mundo todo.</span></div></span></div></div></div></h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r267:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r267:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="travelersChoice.coe" aria-labelledby="travelersChoice.coe_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="travelersChoice.coe" value="coe"></div><div class="hTbJa f H o"><label id="travelersChoice.coe_label" for="travelersChoice.coe" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="f Q1 u"><div class="siujn Rb Ce pVgoX"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="M12 3.953a7.442 7.442 0 1 0 .001 14.884A7.442 7.442 0 0 0 12 3.953m0 14.05a6.61 6.61 0 1 1 0-13.218 6.61 6.61 0 0 1 0 13.219M10.343 11.9a.91.91 0 1 1-1.821 0 .91.91 0 0 1 1.821 0m5.134 0a.91.91 0 1 1-1.821 0 .91.91 0 0 1 1.82 0m.82-1.897.84-.913h-1.863A5.8 5.8 0 0 0 12 8.08a5.77 5.77 0 0 0-3.27 1.008H6.862l.84.913a2.567 2.567 0 1 0 3.475 3.78l.823.896.823-.895a2.568 2.568 0 1 0 3.474-3.78m-6.865 3.634a1.738 1.738 0 1 1 0-3.476 1.738 1.738 0 0 1 0 3.476M12 11.85c0-1.143-.832-2.124-1.929-2.543A5 5 0 0 1 12 8.92a5 5 0 0 1 1.928.386c-1.096.42-1.927 1.4-1.927 2.543m2.566 1.787a1.738 1.738 0 1 1 .001-3.476 1.738 1.738 0 0 1 0 3.476m-8.456 3.719s-.377-.946-1.396-1.903c-1.02-.957-2.303-1.132-2.303-1.132s.457 1.02 1.54 2.04c1.086 1.017 2.159.995 2.159.995m2.568 1.41s-.524-.511-1.479-.883-1.861-.191-1.861-.191.598.54 1.615.935c1.016.397 1.725.139 1.725.139m2.493.505s-.545-.224-1.357-.196-1.415.47-1.415.47.608.222 1.473.193 1.3-.467 1.3-.467m-6.186-4.203s-.175-1.008-.974-2.154c-.8-1.147-2.015-1.578-2.015-1.578s.238 1.098 1.089 2.319c.85 1.22 1.9 1.413 1.9 1.413m-1.003-3.071s.195-1.021-.134-2.393c-.328-1.371-1.294-2.21-1.294-2.21s-.17 1.128.18 2.589c.35 1.46 1.248 2.014 1.248 2.014"></path><path d="M17.887 17.355s.377-.946 1.396-1.903c1.02-.957 2.303-1.132 2.303-1.132s-.457 1.02-1.54 2.04c-1.086 1.017-2.159.995-2.159.995m-2.567 1.41s.524-.511 1.479-.883 1.861-.191 1.861-.191-.598.54-1.615.935c-1.016.397-1.725.139-1.725.139m-2.493.505s.545-.224 1.357-.196 1.415.47 1.415.47-.608.222-1.473.193-1.3-.467-1.3-.467m6.186-4.203s.175-1.008.974-2.154c.8-1.147 2.015-1.578 2.015-1.578s-.238 1.098-1.089 2.319c-.85 1.22-1.9 1.413-1.9 1.413m1.003-3.071s-.195-1.021.133-2.393c.33-1.371 1.293-2.21 1.293-2.21s.17 1.128-.18 2.589c-.349 1.46-1.246 2.014-1.246 2.014M12 20.047a.413.413 0 1 0 0-.827.413.413 0 0 0 0 .827"></path></svg></div>Travellers’ Choice</div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Preço</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r26a:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26a:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="priceTypes.10953" aria-labelledby="priceTypes.10953_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="priceTypes.10953" value="10953"></div><div class="hTbJa f H o"><label id="priceTypes.10953_label" for="priceTypes.10953" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Comer barato</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="priceTypes.10955" aria-labelledby="priceTypes.10955_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="priceTypes.10955" value="10955"></div><div class="hTbJa f H o"><label id="priceTypes.10955_label" for="priceTypes.10955" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Intermediário</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="priceTypes.10954" aria-labelledby="priceTypes.10954_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="priceTypes.10954" value="10954"></div><div class="hTbJa f H o"><label id="priceTypes.10954_label" for="priceTypes.10954" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Restaurante sofisticado</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Pontuação</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-r26e:" class="FSCFM _S B- Cl G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26e:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="minimumTravelerRating.TRAVELER_RATING_LOW" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_LOW_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_LOW" value="TRAVELER_RATING_LOW"></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_LOW_label" for="minimumTravelerRating.TRAVELER_RATING_LOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-r26g:" data-automation="bubbleRatingImage"><title id=":lithium-r26g:">3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P pZUbB biKBZ AWdfh">ou mais</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="minimumTravelerRating.TRAVELER_RATING_MEDIUM" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_MEDIUM" value="TRAVELER_RATING_MEDIUM"></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" for="minimumTravelerRating.TRAVELER_RATING_MEDIUM" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-r26i:" data-automation="bubbleRatingImage"><title id=":lithium-r26i:">4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P pZUbB biKBZ AWdfh">ou mais</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="minimumTravelerRating.TRAVELER_RATING_HIGH" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_HIGH_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_HIGH" value="TRAVELER_RATING_HIGH"></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_HIGH_label" for="minimumTravelerRating.TRAVELER_RATING_HIGH" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-r26k:" data-automation="bubbleRatingImage"><title id=":lithium-r26k:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg></div></div></div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Aberto agora</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r26l:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26l:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="openNow.OPEN_NOW" aria-labelledby="openNow.OPEN_NOW_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="openNow.OPEN_NOW" value="OPEN_NOW"></div><div class="hTbJa f H o"><label id="openNow.OPEN_NOW_label" for="openNow.OPEN_NOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Aberto agora</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Restrições alimentares</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r26n:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26n:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diets.10665" aria-labelledby="diets.10665_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diets.10665" value="10665"></div><div class="hTbJa f H o"><label id="diets.10665_label" for="diets.10665" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Adequado para vegetarianos</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diets.10697" aria-labelledby="diets.10697_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diets.10697" value="10697"></div><div class="hTbJa f H o"><label id="diets.10697_label" for="diets.10697" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Opções veganas</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diets.10992" aria-labelledby="diets.10992_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diets.10992" value="10992"></div><div class="hTbJa f H o"><label id="diets.10992_label" for="diets.10992" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Opções sem glúten</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Excelente para</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r26r:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r26r:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10604" aria-labelledby="styles.10604_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10604" value="10604"></div><div class="hTbJa f H o"><label id="styles.10604_label" for="styles.10604" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Famílias com crianças</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10609" aria-labelledby="styles.10609_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10609" value="10609"></div><div class="hTbJa f H o"><label id="styles.10609_label" for="styles.10609" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Grupos grandes</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10605" aria-labelledby="styles.10605_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10605" value="10605"></div><div class="hTbJa f H o"><label id="styles.10605_label" for="styles.10605" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Reunião de negócios</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="styles.10608" aria-labelledby="styles.10608_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="styles.10608" value="10608"></div><div class="hTbJa f H o"><label id="styles.10608_label" for="styles.10608" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Atmosfera</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button" aria-expanded="false" aria-controls="filter-expando-styles"><span class="biGQs _P ezezH">Mostrar mais<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Características</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r270:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r270:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.16547" aria-labelledby="diningOptions.16547_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.16547" value="16547"></div><div class="hTbJa f H o"><label id="diningOptions.16547_label" for="diningOptions.16547" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Serviço de mesa</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.10852" aria-labelledby="diningOptions.10852_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.10852" value="10852"></div><div class="hTbJa f H o"><label id="diningOptions.10852_label" for="diningOptions.10852" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Lugares para sentar</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.10601" aria-labelledby="diningOptions.10601_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.10601" value="10601"></div><div class="hTbJa f H o"><label id="diningOptions.10601_label" for="diningOptions.10601" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Para levar</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="diningOptions.10861" aria-labelledby="diningOptions.10861_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="diningOptions.10861" value="10861"></div><div class="hTbJa f H o"><label id="diningOptions.10861_label" for="diningOptions.10861" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Acesso para cadeirantes</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W _T c G_ bYExr wnNQG LbPqG" type="button"><span class="biGQs _P ezezH">Mostrar tudo</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P fiohW ezezH">Bairro</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-r275:" class="FSCFM _S B- Cl G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-r275:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945488" aria-labelledby="neighbourhoods.20945488_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945488" value="20945488"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945488_label" for="neighbourhoods.20945488" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Umarizal</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945487" aria-labelledby="neighbourhoods.20945487_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945487" value="20945487"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945487_label" for="neighbourhoods.20945487" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Nazare</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945485" aria-labelledby="neighbourhoods.20945485_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945485" value="20945485"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945485_label" for="neighbourhoods.20945485" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Campina</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="opEmP"><input id="neighbourhoods.20945486" aria-labelledby="neighbourhoods.20945486_label" type="checkbox" class="PMWyE _G _S R2 Vm _T bYExr" aria-checked="false" name="neighbourhoods.20945486" value="20945486"></div><div class="hTbJa f H o"><label id="neighbourhoods.20945486_label" for="neighbourhoods.20945486" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P pZUbB AWdfh">Fatima</span></span></span></label></div></span></div></div></div></div></div></span></div></div><div class="oqGNy" data-automation="LeftRailSlot"><div class="uGSLz f e"><div class="IUxgK"><div class="CsVUk E" style="top: 24px;"><div class="YcOBw mFLzV gGJkl"><div class="Eufjb j u" id="slot:300x600:rail1" data-google-query-id="CILs0Kzq2I4DFbY5uQYd5NY64A"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_38__container__" style="border: 0pt none; display: inline-block; width: 300px; height: 600px;"><iframe frameborder="0" src="https://f317dcf7d6522885a606297e1d4c070c.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_38" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="300" height="600" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Advertisement" tabindex="0" data-google-container-id="a" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></div></div></div></div><div class="IUxgK"><div class="CsVUk E" style="top: 24px;"><div class="YcOBw mFLzV gGJkl"><div class="Eufjb j u ChFsW Xd o S" id="slot:300x600:rail3"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_39__container__" style="border: 0pt none; display: inline-block;"></div></div></div></div></div></div></div></div><div class="LvzMf _T" data-automation="LeftRailMain"><div class="u f Q2"><div class="biGQs _P pZUbB AWdfh" data-automation="resultsTotal">1.096 resultados</div><div class="NB"><span><div class="f u"><div class="biGQs _P pZUbB egaXP kSNRl ZNjnF">Ordenar por: </div><div class="lOvTR"><button class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" type="button"><span class="biGQs _P ezezH">Em destaque</span><div class="YQrIS _W PH"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div><div class="DgJpu"><button type="button" class="ypcsE _S bYExr" aria-labelledby=":lithium-r27b:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb rihVH UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-r27b:" class="KrpDA o">Restaurantes classificados de acordo com visualizações de página, avaliações e atributos individuais, como faixa de preço, culinária e localização, bem como dados agregados do Tripadvisor que comparam as interações dos usuários com todas as outras páginas de restaurantes neste local.</span></div></div></span></div></div><div class="fKQIr"><div class="rfqBq f e"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">1. Restaurante Ver O Açaí - Umarizal</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r27i:" data-automation="bubbleRatingImage"><title id=":lithium-r27i:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Bar</span></span><span class="biGQs _P pZUbB ZNjnF"></span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://veroacai.com.br/lp/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sensacional</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale a experiencia </span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/9f/ab/24/menu-degustacao-paraense.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/fa/e2/d2/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/d8/09/e1/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-23588946"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">2. Amazônia Na Cuia</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r27p:" data-automation="bubbleRatingImage"><title id=":lithium-r27p:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(92 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://pedidosup.com.br/amazonianacuiadelivery/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Belém</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Muito bom</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="Campeão de venda chapa mista paraense com açaí." loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="pirarucu com açaí" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/e7/88/c9/pirarucu-com-acai.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=700&amp;h=700&amp;s=1 2x"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="Fotos pratos do Point" loading="lazy" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0a/1b/2e/da/fotos-pratos-do-point.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6483240"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">3. Restaurante Point do Açaí</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r280:" data-automation="bubbleRatingImage"><title id=":lithium-r280:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(4.884 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc bYExr jWkoZ QHaGY VauJM" href="/UserReviewEdit-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div><div class=""><div id="slot:728x90:inline3" class="Eufjb j u nFcpI" data-google-query-id="CPKbzqzq2I4DFbY5uQYd5NY64A"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_40__container__" style="border: 0pt none; display: inline-block; width: 100%; height: auto;"><iframe frameborder="0" src="https://f317dcf7d6522885a606297e1d4c070c.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_40" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="0" height="190" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Advertisement" tabindex="0" data-google-container-id="c" style="border: 0px; vertical-align: bottom; min-width: 100%;" data-load-complete="true"></iframe></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373246"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">4. Manjar das Garças</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r287:" data-automation="bubbleRatingImage"><title id=":lithium-r287:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.206 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa comida, mas com preço elevado</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente programa.</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2360313"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">5. Remanso do Peixe</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r28e:" data-automation="bubbleRatingImage"><title id=":lithium-r28e:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(915 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Saboroso</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Remanseando no Remanso!</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/9e/75/b4/photo5jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-19450192"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">6. Casa Do Saulo Das Onze Janelas</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r28l:" data-automation="bubbleRatingImage"><title id=":lithium-r28l:">4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(166 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Contemporânea</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="http://linklist.bio/casadosaulo" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale a visita apesar de.....</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d19450192-Reviews-Casa_Do_Saulo_Das_Onze_Janelas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale sim a visita </span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25365698"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">7. Santa Grelha Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r28s:" data-automation="bubbleRatingImage"><title id=":lithium-r28s:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(182 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Steakhouse</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente lugar</span></div></a></div></div></div></div></div></div><div class="LthNE qWPrE ncFvv" data-automation="shelf"><hr class="RMdFe tyUdl"><span><div class="vqnnc"><div class="hHadL f k Q3 u MJ"><div class="biGQs _P fiohW KeZJf">Restaurantes sofisticados</div><a class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" href="/Restaurants-g303404-zfp10954-Belem_State_of_Para.html" data-automation="seeAllLink" target="_blank"><span class="biGQs _P ezezH">Veja tudo</span></a></div><div class="aEwIm"><div class="YRcYv f Q3 WDFZm _h"><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="323" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Umarizal</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r293:" data-automation="bubbleRatingImage"><title id=":lithium-r293:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25377098"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Ryori belém</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29a:" data-automation="bubbleRatingImage"><title id=":lithium-r29a:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/27/a9/3d/9e/sempre-renovando-nossa.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373223"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Famiglia Sicilia</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29h:" data-automation="bubbleRatingImage"><title id=":lithium-r29h:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(451 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Italiana • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="301" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373246"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Manjar das Garcas</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29o:" data-automation="bubbleRatingImage"><title id=":lithium-r29o:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.206 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0b/8d/32/ad/images-1-largejpg.jpg?w=300&amp;h=-1&amp;s=1" width="300" height="250" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373270"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Grand Cru</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r29v:" data-automation="bubbleRatingImage"><title id=":lithium-r29v:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(463 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Europeia • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6009425"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sushi Ruy Barbosa</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2a6:" data-automation="bubbleRatingImage"><title id=":lithium-r2a6:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d6009066-Reviews-Santa_Chicoria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/84/f7/18/photo3jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6009066"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Santa Chicoria</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ad:" data-automation="bubbleRatingImage"><title id=":lithium-r2ad:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d6009066-Reviews-Santa_Chicoria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(188 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d25846365-Reviews-Zu_Gourmande-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/82/6a/87/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25846365"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Zu Gourmande</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ak:" data-automation="bubbleRatingImage"><title id=":lithium-r2ak:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d25846365-Reviews-Zu_Gourmande-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(22 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Francesa • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d23445371-Reviews-Purao_Vegano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/81/01/2c/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-23445371"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Purão Vegano</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ar:" data-automation="bubbleRatingImage"><title id=":lithium-r2ar:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d23445371-Reviews-Purao_Vegano-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(11 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d3507098-Reviews-A_Forneria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/99/4f/73/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3507098"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">A Forneria</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>3,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2b2:" data-automation="bubbleRatingImage"><title id=":lithium-r2b2:">3,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d3507098-Reviews-A_Forneria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(136 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Italiana • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div></div><div class="zOqpq u f S t w"><div class="nxlXU S CxuXo"></div><div class="iWPJm u k l _Q w Z1"><div class="eaUlz H"></div><div class="HFJew"><button class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" type="button" aria-label="Avançar"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div></div></div></div></div></span><hr class="RMdFe roAGK"></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/e0/e4/b1/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25377098"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">8. Ryori belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2b9:" data-automation="bubbleRatingImage"><title id=":lithium-r2b9:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d25377098-Reviews-Ryori_belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa experiência de japonês</span></div></a></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline4" class="Eufjb j u ChFsW Xd o S nFcpI"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_41__container__" style="border: 0pt none; display: inline-block;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-27765971"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">9. Restaurante Ver O Açaí - Marco</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2bg:" data-automation="bubbleRatingImage"><title id=":lithium-r2bg:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(339 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://veroacai.com.br/lp/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Comida e atendimento por excelência.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Experimente a Amazônia em um lugar</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24040497"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">10. Camarada Camarão Belém - Shopping Boulevard Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2bn:" data-automation="bubbleRatingImage"><title id=":lithium-r2bn:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(1.698 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://livemenu.app/menu/627bcbac218bd50011827877" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Camarada.</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-13437655"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">11. Geek Burger</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2bu:" data-automation="bubbleRatingImage"><title id=":lithium-r2bu:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(375 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Americana, Bar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://linktr.ee/geekburgeroficial" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Divertido</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/cf/8a/a6/salao-2.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3849186"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">12. Govinda Restaurante Vegetariano</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,7</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2c5:" data-automation="bubbleRatingImage"><title id=":lithium-r2c5:">4,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(303 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Indiana, Saudável</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://micrositios.ifood.com.br/delivery/belem-pa/govinda-restaurante-vegetariano-batista-campos/56e69336-66ec-42f7-ab96-43d70c523be1" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Não sou vegana, mas amo a experiência gastronômica.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3849186-Reviews-Govinda_Vegetariano-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Govinda vegetariano Belém</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=700&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3259642"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">13. Saldosa Maloca</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2cc:" data-automation="bubbleRatingImage"><title id=":lithium-r2cc:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(316 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Experiência.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sabores do Pará.</span></div></a></div></div></div></div></div></div><div class="LthNE qWPrE ncFvv" data-automation="shelf"><hr class="RMdFe tyUdl"><span><div class="vqnnc"><div class="hHadL f k Q3 u MJ"><div class="biGQs _P fiohW KeZJf">Restaurantes locais</div><a class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" href="/Restaurants-g303404-zft10613-Belem_State_of_Para.html" data-automation="seeAllLink" target="_blank"><span class="biGQs _P ezezH">Veja tudo</span></a></div><div class="aEwIm"><div class="YRcYv f Q3 WDFZm _h"><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="323" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Umarizal</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2cj:" data-automation="bubbleRatingImage"><title id=":lithium-r2cj:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/81/e8/a5/campeao-de-venda-chapa.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6483240"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Point do Açai</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2cq:" data-automation="bubbleRatingImage"><title id=":lithium-r2cq:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d6483240-Reviews-Restaurante_Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(4.884 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-26839232"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Ver-o-Pesinho</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2d1:" data-automation="bubbleRatingImage"><title id=":lithium-r2d1:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(299 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-27765971"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Marco</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2d8:" data-automation="bubbleRatingImage"><title id=":lithium-r2d8:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(339 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/ad/a1/11/moqueca-paraense-do-seu.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2360313"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Remanso do Peixe</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2df:" data-automation="bubbleRatingImage"><title id=":lithium-r2df:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2360313-Reviews-Remanso_do_Peixe-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(915 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=900&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/c0/5c/b5/photo2jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="301" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373246"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Manjar das Garcas</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2dm:" data-automation="bubbleRatingImage"><title id=":lithium-r2dm:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373246-Reviews-Manjar_das_Garcas-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.206 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3902846"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Cervejaria Amazon Beer</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2dt:" data-automation="bubbleRatingImage"><title id=":lithium-r2dt:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.488 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=700&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0e/38/c0/8c/photo4jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3259642"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Saldosa Maloca</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2e4:" data-automation="bubbleRatingImage"><title id=":lithium-r2e4:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d3259642-Reviews-Saldosa_Maloca-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(316 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-12895755"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Point do Acai</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,7</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2eb:" data-automation="bubbleRatingImage"><title id=":lithium-r2eb:">4,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(159 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373303"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Tacaca da Dona Maria</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ei:" data-automation="bubbleRatingImage"><title id=":lithium-r2ei:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(287 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $</div></div></div></a></span></div></div><div class="zOqpq u f S t w"><div class="nxlXU S CxuXo"></div><div class="iWPJm u k l _Q w Z1"><div class="eaUlz H"></div><div class="HFJew"><button class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" type="button" aria-label="Avançar"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div></div></div></div></div></span><hr class="RMdFe roAGK"></div><div class=""><div id="slot:728x90:inline5" class="Eufjb j u ChFsW Xd o S nFcpI"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_42__container__" style="border: 0pt none; display: inline-block;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-19246676"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">14. Sushi Boulevard</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ep:" data-automation="bubbleRatingImage"><title id=":lithium-r2ep:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(497 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Sushi</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc bYExr jWkoZ QHaGY VauJM" href="/UserReviewEdit-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e4/6b/4c/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-12895755"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Vencedor do Travellers’ Choice 2024" data-automation="travelersChoice-2024"><img src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2024/2024_TC_Badge_Large.svg" class="TPuys zVLQy YkUfS" aria-hidden="true"></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">15. Point do Açaí</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,7</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2f0:" data-automation="bubbleRatingImage"><title id=":lithium-r2f0:">4,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(159 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Visita gastronômica</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d12895755-Reviews-Point_do_Acai-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Luciano: garçom incrível.</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1b/87/4f/15/unha-de-caranguejo.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3902846"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">16. Cervejaria Amazon Beer</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2f7:" data-automation="bubbleRatingImage"><title id=":lithium-r2f7:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.488 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Pub com cerveja artesanal</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://pedidosup.com.br/amazonbeercardapio/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Cervejar e petiscar</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3902846-Reviews-Cervejaria_Amazon_Beer-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Chopps especiais</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=700&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-10158580"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">17. Sushi Boulevard</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2fe:" data-automation="bubbleRatingImage"><title id=":lithium-r2fe:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(790 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Sushi</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="http://www.sushiboulevard.com.br/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc bYExr jWkoZ QHaGY VauJM" href="/UserReviewEdit-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/12/f5/ac/52/ta-img-20180515-201010.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/12/f5/ac/52/ta-img-20180515-201010.jpg?w=700&amp;h=700&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/12/f5/ac/52/ta-img-20180515-201010.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-4574569"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">18. Xicara da Silva</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2fl:" data-automation="bubbleRatingImage"><title id=":lithium-r2fl:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(539 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Pizza</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d4574569-Reviews-Xicara_da_Silva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Jantar</span></div></a></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline6" class="Eufjb j u ChFsW Xd o S nFcpI"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_43__container__" style="border: 0pt none; display: inline-block;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4e/9b/55/roxy-bar-o-prazer-de.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-857993"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">19. Roxy Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2fs:" data-automation="bubbleRatingImage"><title id=":lithium-r2fs:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(1.584 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vá que é bom.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d857993-Reviews-Roxy_Bar-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Tudo muito bom! Recomendamos! </span></div></a></div></div></div></div></div></div><div class="LthNE qWPrE ncFvv" data-automation="shelf"><hr class="RMdFe tyUdl"><span><div class="vqnnc"><div class="hHadL f k Q3 u MJ"><div class="biGQs _P fiohW KeZJf">Restaurantes acessíveis</div><a class="xyffN _G B- _S _W _T c G_ bYExr eeawa cIzpA" href="/Restaurants-g303404-zfp10955-Belem_State_of_Para.html" data-automation="seeAllLink" target="_blank"><span class="biGQs _P ezezH">Veja tudo</span></a></div><div class="aEwIm"><div class="YRcYv f Q3 WDFZm _h"><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/46/22/92/unidade-umarizal.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="323" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-16751056"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Umarizal</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2g3:" data-automation="bubbleRatingImage"><title id=":lithium-r2g3:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.529 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/74/b0/e8/chopes-e-petisco-praia.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24040497"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Camarada Camarão Belém - Shopping Boulevard Belém</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ga:" data-automation="bubbleRatingImage"><title id=":lithium-r2ga:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(1.698 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1400&amp;h=1000&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/da/b8/89/temos-camisas-livros.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-26839232"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Ver-o-Pesinho</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2gh:" data-automation="bubbleRatingImage"><title id=":lithium-r2gh:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d26839232-Reviews-Ver_o_Pesinho-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(299 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=700&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1400&amp;h=-1&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1600&amp;h=-1&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=500&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1000&amp;h=-1&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=600&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=1200&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/9c/a6/restaurante-ver-o-acai.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-27765971"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Restaurante Ver O Açaí - Marco</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2go:" data-automation="bubbleRatingImage"><title id=":lithium-r2go:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d27765971-Reviews-Restaurante_Ver_O_Acai_Marco-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(339 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24995218"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Coco Bambu Belém</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2gv:" data-automation="bubbleRatingImage"><title id=":lithium-r2gv:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(950 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/21/65/85/atualizacao-da-fachada.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-13437655"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Geek Burger</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2h6:" data-automation="bubbleRatingImage"><title id=":lithium-r2h6:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d13437655-Reviews-Geek_Burger-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(375 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Americana • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=1100&amp;h=800&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/e8/0e/8b/faca-seu-evento-conosco.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25365698"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">SANTA GRELHA BELÉM</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2hd:" data-automation="bubbleRatingImage"><title id=":lithium-r2hd:">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d25365698-Reviews-SANTA_GRELHA_BELEM-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(182 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Steakhouse • $$ - $$$</div><div class="jMJSs"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Aberto agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/6e/a8/fd/caption.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-19246676"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sushi Boulevard</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2hk:" data-automation="bubbleRatingImage"><title id=":lithium-r2hk:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d19246676-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(497 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=700&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/54/5b/59/super-combinado.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-10158580"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sushi Boulevard</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2hr:" data-automation="bubbleRatingImage"><title id=":lithium-r2hr:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d10158580-Reviews-Sushi_Boulevard-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(790 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Japonesa • $$ - $$$</div><div class="lKjIZ"><div class="biGQs _P alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Fechado agora</div></div></div></div></a></span></div><div class="mzZhu I CxuXo"><span><a target="_blank" href="/Restaurant_Review-g303404-d7747107-Reviews-Sorveteria_Cairu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ keSJi"><div class="oCArq e Q1 _Z"><div class="hIxoW MH o w"><div class="afQPz SyjMH qcDHf o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="FGwzt PaRlG"><div style="aspect-ratio: 1.33333 / 1;"><picture class="NhWcC _R mdkdE o afQPz SyjMH ttWhi"><source media="(max-width: 400px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 500px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 600px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1200&amp;h=900&amp;s=1 2x"><source media="(max-width: 700px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=700&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1400&amp;h=1100&amp;s=1 2x"><source media="(max-width: 767px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1600&amp;h=1200&amp;s=1 2x"><source media="(max-width: 800px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 2x"><source media="(max-width: 1000px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=500&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1000&amp;h=800&amp;s=1 2x"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=600&amp;h=500&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=1200&amp;h=900&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=800&amp;h=600&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/fa/98/f8/sorveteria-cairu.jpg?w=400&amp;h=300&amp;s=1" width="400" height="300" alt="" loading="lazy"></picture></div></div></div><div class="cnQjJ z REnnF"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-7747107"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div><div class="ZvrsW N G"><div><div class="odsgl Q1"></div></div><div class="ZvrsW N G"><div class="biGQs _P fiohW OgHoE">Sorveteria Cairu</div><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2i2:" data-automation="bubbleRatingImage"><title id=":lithium-r2i2:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_blank" href="/Restaurant_Review-g303404-d7747107-Reviews-Sorveteria_Cairu-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(2.250 avaliações) </span></div></a></div></div></div></div></div></div><div class="ZvrsW N G"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W ZNjnF">Brasileira • $$ - $$$</div></div></div></a></span></div></div><div class="zOqpq u f S t w"><div class="nxlXU S CxuXo"></div><div class="iWPJm u k l _Q w Z1"><div class="eaUlz H"></div><div class="HFJew"><button class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" type="button" aria-label="Avançar"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div></div></div></div></div></span><hr class="RMdFe roAGK"></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/b8/bf/6a/lagosta-imperial.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-24995218"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">20. Coco Bambu Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2i9:" data-automation="bubbleRatingImage"><title id=":lithium-r2i9:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(950 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Café</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Agregou mais valor a nossa viagem 😍</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d24995218-Reviews-Coco_Bambu_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Comida e atendimento excelentes</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/24/fa/cb/29/fachada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-23315259"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">21. Tap House Boteco Ver-a-Cerva</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2ig:" data-automation="bubbleRatingImage"><title id=":lithium-r2ig:">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(60 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Bares e pubs, Americana</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Cervejas e Drinks em Belém</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d23315259-Reviews-Tap_House_Boteco_Ver_a_Cerva-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sensacional</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/27/a9/3d/9e/sempre-renovando-nossa.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373223"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">22. Famiglia Sicilia</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2in:" data-automation="bubbleRatingImage"><title id=":lithium-r2in:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(451 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Italiana, Europeia</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Uma experiência surpreendente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373223-Reviews-Famiglia_Sicilia-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Italiano paraense de respeito</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/01/f3/65/fachaada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-25432845"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">23. Tap House Ver-A-Cerva - Batista Campos</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2iu:" data-automation="bubbleRatingImage"><title id=":lithium-r2iu:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(38 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Bares e pubs, Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://veracerva.com.br/lp/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Otimo local para tomar cerveja e ouvir excelentes músicas, con atendimento muito atencioso</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d25432845-Reviews-Tap_House_Ver_A_Cerva_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Gostoso!</span></div></a></div></div></div></div></div></div><div data-automation="nova-ad-InlineNova"><div class="VnzpY"><div id="slot:728x90-3x3:InlineNova" class="RnYjA j u ZWKar Xd y o"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_44__container__" style="border: 0pt none; display: inline-block;"></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/30/07/c8/esperando-voces-pra-fazer.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-8624369"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">24. Porpino Burger</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,6</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2j5:" data-automation="bubbleRatingImage"><title id=":lithium-r2j5:">4,6 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(833 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Americana, Bar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P pZUbB ZNjnF"><span class="f"><a target="_blank" href="https://porpinoburger.com/menu/" class="BMQDV _F Gv bYExr SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d8624369-Reviews-Porpino_Burger_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Maravilhoso!</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/97/2b/0b/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-8826462"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">25. restaurante chalé da Ilha do combu.</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,2</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2jc:" data-automation="bubbleRatingImage"><title id=":lithium-r2jc:">4,2 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(118 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Bar</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Estrutura magnífica, serviço lento</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d8826462-Reviews-Restaurante_chale_da_Ilha_do_combu-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Um dos melhores bares/restaurantes da Ilha</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/4c/d5/67/reforma-do-sushi-ruy.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-6009425"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">26. Sushi Ruy Barbosa</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2jj:" data-automation="bubbleRatingImage"><title id=":lithium-r2jj:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(329 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Japonesa, Sushi</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Muito bom, mas caro.</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d6009425-Reviews-Sushi_Ruy_Barbosa-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Ambiente muito agradável</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/51/8d/b5/photo1jpg.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373303"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">27. Tacacá da Dona Maria</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2jq:" data-automation="bubbleRatingImage"><title id=":lithium-r2jq:">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(287 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira</span></span><span class="biGQs _P pZUbB ZNjnF">$</span></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">No centro de Belém</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373303-Reviews-Tacaca_da_Dona_Maria-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">delicioso</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/d2/72/b7/la-traviata.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-3193396"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">28. La Traviata</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,3</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2k1:" data-automation="bubbleRatingImage"><title id=":lithium-r2k1:">4,3 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(532 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Italiana, Pizza</span></span><span class="biGQs _P pZUbB ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Dicas para o proprietario</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d3193396-Reviews-La_Traviata-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">O forte é a comida italiana</span></div></a></div></div></div></div></div></div><div class=""><div class="NxBlo Na"><a target="_blank" href="https://www.tripadvisor.com.br/Articles" class="BMQDV _F Gv bYExr SwZTJ hNpWR"><section aria-label="Continue planejando O que fazer, onde comer e mais inspirações de viagem. Veja mais" class="BQTWL f zQRfF"><div class="Bsnup f"><div class="dsJsk o"><picture class="NhWcC _R afQPz eXZKw"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/2d/40/95/keep-on-planning.jpg?w=200&amp;h=200&amp;s=1&amp;cx=492&amp;cy=358&amp;chk=v1_f64df95338d294710736 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/2d/40/95/keep-on-planning.jpg?w=400&amp;h=400&amp;s=1&amp;cx=492&amp;cy=358&amp;chk=v1_f64df95338d294710736 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/2d/40/95/keep-on-planning.jpg?w=200&amp;h=200&amp;s=1&amp;cx=492&amp;cy=358&amp;chk=v1_f64df95338d294710736" width="200" height="200" alt="" loading="lazy"></picture></div></div><div class="GiBeo o F4 f"><div class="LVEbf f"><div class=""><div class="biGQs _P fiohW alXOW EEXWj GzNcM BYtua UTQMg alvrA KeZJf">Continue planejando</div></div><div class=""><div class="biGQs _P fiohW alXOW EEXWj GzNcM BYtua UTQMg alvrA AWdfh">O que fazer, onde comer e mais inspirações de viagem.</div></div></div><div class="mJfQs f"><div class="CwwAT"><button class="rmyCe _G B- z _S c Wc bYExr w AeLHi QHaGY VauJM" type="button"><span class="biGQs _P ezezH">Veja mais</span></button></div></div></div></section></a></div><div id="slot:728x90:inline7" class="Eufjb j u ChFsW Xd o S nFcpI" data-google-query-id="CPebzqzq2I4DFbY5uQYd5NY64A" style="display: none;"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_45__container__" style="border: 0pt none; width: 728px; height: 0px;"></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><img src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0b/8d/32/ad/images-1-largejpg.jpg?w=300&amp;h=300&amp;s=1" width="300" height="250" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373270"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">29. Grand Cru</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,4</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2k8:" data-automation="bubbleRatingImage"><title id=":lithium-r2k8:">4,4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(463 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Bares e pubs, Europeia</span></span><span class="biGQs _P pZUbB ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Ambiente muito agradável</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373270-Reviews-Grand_Cru-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vinhos excitantes</span></div></a></div></div></div></div></div></div><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T Fl y"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc JISbF bYExr" data-automation="photoCarousel"><div class="yMdQy w iwPdm"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/21/e8/65/d5/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" alt="" loading="lazy" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S bUoBZ HuPlH gSpCh IyzRb xtNBb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S bUoBZ HuPlH IyzRb xtNBb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F BaKrP xwbQG cunPs jODdB" data-automation="trips-save-button-item-15273271"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y"><div class="IcVzi y _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P fiohW alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">30. UATA?!</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-r2kf:" data-automation="bubbleRatingImage"><title id=":lithium-r2kf:">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P pZUbB ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv bYExr SwZTJ FGwzt suezE"><div class="biGQs _P pZUbB ZNjnF" data-automation="bubbleReviewCount"><span>(144 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P pZUbB ZNjnF">Brasileira, Fast food</span></span><span class="biGQs _P pZUbB ZNjnF">$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Abre em 41 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Ótimos, sorridentes e super competentes os atendimentos</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d15273271-Reviews-UATA-Belem_State_of_Para.html" class="BMQDV _F Gv bYExr SwZTJ FGwzt ukgoS"><div class="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amei o atendimento</span></div></a></div></div></div></div></div></div><div class="qAZoU c"><div class="GMYGA"><div class="mkNRT j"><div class="HXmEd"><div class="IGLCo"><a class="BrOJk u j z _F _S bYExr tIqAi unMkR xtNBb" data-smoke-attr="pagination-next-arrow" aria-label="Próxima página" href="/Restaurants-g303404-oa30-Belem_State_of_Para.html"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></a></div></div></div><div class="LaMdn j"><div class="aMnVW"><div class="Yzhnw P"><button class="BrOJk u j z _F _S bYExr tIqAi iNBVo SSqtP xtNBb" type="button" disabled="" aria-label="1"><span class="kLqdM"><span class="biGQs _P ezezH">1</span></span></button></div><div class="Yzhnw P"><a class="BrOJk u j z _F _S bYExr tIqAi iNBVo xtNBb" aria-label="2" href="/Restaurants-g303404-oa30-Belem_State_of_Para.html"><span class="kLqdM"><span class="biGQs _P ezezH">2</span></span></a></div><div class="Yzhnw P"><div class="biGQs _P fiohW ezezH"><div class="sUgyD">…</div></div></div></div></div></div></div><div class="qAZoU c"><div class="biGQs _P pZUbB ZNjnF"><div class="Ci">Mostrando 1–30 de 1.096 resultados</div></div></div></div></div></div></div></div></div>
"""

soup_tripadvisor = BeautifulSoup(html_do_tripadvisor, 'html.parser')

print("\nIniciando a extração detalhada dos restaurantes da página do Tripadvisor...")
restaurantes_detalhados = [] 

restaurant_blocks = soup_tripadvisor.find_all('div', class_=lambda c: c and 'tbrcR' in c)

if not restaurant_blocks:
    print("AVISO: Nenhum bloco de restaurante principal encontrado com os seletores configurados ('tbrcR').")
    print("Por favor, **REVISE A INSPEÇÃO MANUAL** se o layout da página mudou e atualize este seletor.")

top_n_restaurants = restaurant_blocks[:5] 

for block in top_n_restaurants:

    nome_link_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'ukgoS' in c)
    nome = "Nome não encontrado"
    if nome_link_tag:
        nome_div_tag = nome_link_tag.find('div', class_=lambda c: c and 'biGQs' in c and 'fiohW' in c)
        if nome_div_tag:
            nome = nome_div_tag.get_text(strip=True)

    reviews_tag = block.find('div', {'data-automation': 'bubbleReviewCount'})
    reviews = reviews_tag.find('span').get_text(strip=True) if reviews_tag and reviews_tag.find('span') else "Reviews não encontrados"


    rating_tag = block.find('div', {'data-automation': 'bubbleRatingValue'})
    rating = rating_tag.find('span').get_text(strip=True) if rating_tag and rating_tag.find('span') else "Avaliação não encontrada"


    culinaria_preco_div = block.find('div', class_=lambda c: c and 'ZvrsW' in c and 'biqBm' in c)

    tipo_culinaria = "Tipo de Culinária não encontrado"
    preco = "Preço não encontrado"

    if culinaria_preco_div:

        spans_info = culinaria_preco_div.find_all('span', class_=lambda c: c and 'biGQs' in c and 'pZUbB' in c and 'ZNjnF' in c)


        if len(spans_info) >= 1:

            tipo_culinaria = spans_info[0].get_text(strip=True)


            for i in range(1, len(spans_info)):
                text = spans_info[i].get_text(strip=True)
                if '$' in text: 
                    preco = text
                    break 


    localizacao = "Localização não especificada (do nome)"
    if ' - ' in nome:

        partes_nome = nome.split(' - ')
        if len(partes_nome) > 1:
            localizacao = partes_nome[-1].strip()


    link = "Link não encontrado"
    if nome_link_tag and 'href' in nome_link_tag.attrs:
        link = "https://www.tripadvisor.com.br" + nome_link_tag['href']


    restaurantes_detalhados.append({
        "Nome": nome,
        "Avaliação": rating,
        "Reviews": reviews,
        "Preço": preco,
        "Tipo Culinária": tipo_culinaria,
        "Localização": localizacao,
        "Link": link
    })


if restaurantes_detalhados:
    print(f"\n--- {len(restaurantes_detalhados)} Restaurantes Extraídos do Tripadvisor ---")
    for i, r in enumerate(restaurantes_detalhados):
        print(f"Restaurante #{i+1}:")
        print(f"  Nome: {r['Nome']}")
        print(f"  Avaliação: {r['Avaliação']}")
        print(f"  Reviews: {r['Reviews']}")
        print(f"  Preço: {r['Preço']}")
        print(f"  Tipo Culinária: {r['Tipo Culinária']}")
        print(f"  Localização: {r['Localização']}")
        print(f"  Link: {r['Link']}")
        print("-" * 40)
else:
    print("Nenhum detalhe de restaurante foi extraído. **Verifique os seletores HTML no Bloco 4**.")

print("-" * 50)


Iniciando a extração detalhada dos restaurantes da página do Tripadvisor...

--- 5 Restaurantes Extraídos do Tripadvisor ---
Restaurante #1:
  Nome: 1. Restaurante Ver O Açaí - Umarizal
  Avaliação: 4,9
  Reviews: (2.529 avaliações)
  Preço: Preço não encontrado
  Tipo Culinária: Brasileira, Bar
  Localização: Umarizal
  Link: https://www.tripadvisor.com.br/Restaurant_Review-g303404-d16751056-Reviews-Restaurante_Ver_O_Acai_Umarizal-Belem_State_of_Para.html
----------------------------------------
Restaurante #2:
  Nome: 2. Amazônia Na Cuia
  Avaliação: 4,4
  Reviews: (92 avaliações)
  Preço: $$ - $$$
  Tipo Culinária: Brasileira, Frutos do mar
  Localização: Localização não especificada (do nome)
  Link: https://www.tripadvisor.com.br/Restaurant_Review-g303404-d23588946-Reviews-Amazonia_Na_Cuia-Belem_State_of_Para.html
----------------------------------------
Restaurante #3:
  Nome: 3. Restaurante Point do Açaí
  Avaliação: 4,8
  Reviews: (4.884 avaliações)
  Preço: $$ - $$$
  Tipo Cu